<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/Assignment-1/Question%203%2C4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
from keras.datasets import fashion_mnist
import numpy as np
from  matplotlib import pyplot as plt
import time
import math
import wandb
wandb.login()





     |████████████████████████████████| 1.7 MB 3.2 MB/s 
     |████████████████████████████████| 144 kB 40.1 MB/s 
     |████████████████████████████████| 180 kB 45.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=b4b67c1a296dc40757428f6f66738bdaf12fd6963e45b825cc8d3c5f7f403a85
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
((xtrain, ytrain), (xtest, ytest)) = fashion_mnist.load_data()

items=["T-shirt/top", "Trouser","Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag","Ankle boot"]
ftrx=np.zeros((60000,784))
ftry=np.zeros((60000,10))
ftex=np.zeros((10000,784))
ftey=np.zeros((10000,10))
for i in range(len(xtrain)):
  ftrx[i]=xtrain[i].flatten();
  ftrx[i]=ftrx[i]/256
  ftry[i][ytrain[i]]=1
for i in range(len(xtest)):
  ftex[i]=xtest[i].flatten();
  ftex[i]=ftex[i]/256
  ftey[i][ytest[i]]=1

def sigmoid(x):
  return np.where(x>=0, 1/(1+np.exp(-x)), np.exp(x)/(1+np.exp(x)))

def grad_sigmoid(z):
  d=sigmoid(z)
  return d*(1-d)

def softmax(z):
  mx=np.max(z)
  z=z-mx
  out=np.exp(z)/np.sum(np.exp(z)+1e-9)
  return out

def grad_softmax(yh,ty):
  return yh-ty

def Relu(x):
    return np.maximum(0, x)

def grad_Relu(x):
    return [1 if xi>0 else 0 for xi in x]

def Tanh(x):
  a= 2*np.where(x>=0, 1/(1+np.exp(-2*x)), np.exp(2*x)/(1+np.exp(2*x)))
  return a-1
def Tanh1(Z):
    mx=np.max(Z)
    #numr = np.exp(x-mx) - np.exp(-x-mx)
    #denom=np.exp(x-mx) + np.exp(-x-mx)
    #mn=np.min(Z)
    # if mn<0:
    #   a=np.exp(Z-mx+mn)
    #   b=np.exp(-Z-mx+mn)
      
    #   #return (a-b+1e-9)/(a+b+1e-8)
    return np.array([((np.exp(x-mx) - np.exp(-x-mx))/((np.exp(x-mx) + np.exp(-x-mx)))) for x in Z])

def grad_Tanh(x):
    return 1-Tanh(x)**2

def InitParams(x,init):
    weights=[]
    baises=[]
    if(init=='random'):
      weights.append((np.random.randn(x[0],784)))
      baises.append((np.random.randn(x[0])))
      l=len(x)-1
      for i in range(l):
          weights.append(0.1*(np.random.randn(x[i+1],x[i])))
          baises.append(0.1*(np.random.randn(x[i+1])))
      weights.append(0.1*(np.random.randn(10,x[-1])))
      baises.append(0.1*(np.random.randn(10)))
    else:
      weights.append(np.random.randn(x[0], 784)*np.sqrt(2./(x[0] + 784)))
      baises.append(np.zeros(x[0]))
      l=len(x)-1
      for i in range(l):
          weights.append(np.random.randn(x[i], x[i-1])*np.sqrt(2./(x[i] + x[i-1])))
          baises.append(np.zeros(x[i+1]))
      weights.append(np.sqrt(2./(10 + x[-1]))*(np.random.randn(10,x[-1])))
      baises.append((np.zeros(10)))


    return weights,baises

def initgrads(N,Nunits):
  dh,da,dw,db=[],[],[],[]
  dw.append(np.zeros((Nunits[0],784)))
  db.append(np.zeros(Nunits[0]))
  da.append(np.zeros(Nunits[0]))
  dh.append(np.zeros(Nunits[0]))
  for i in range(1,N):
    dw.append(np.zeros((Nunits[i],Nunits[i-1])))
    db.append(np.zeros(Nunits[i]))
    da.append(np.zeros(Nunits[i]))
    dh.append(np.zeros(Nunits[i]))
  dw.append(np.zeros((10,Nunits[-1])))
  db.append(np.zeros(10))
  da.append(np.zeros(10))
  dh.append(np.zeros(10))
  return dw,db,da,dh



def train_acc_loss(W,B,activation,ls_fun):
  cost=0
  count=0
  if (ls_fun=='ce'):
    for i in range(len(ftrx[:54000])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost = np.sum(np.multiply(ftry[i],np.log(ypr)))
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return -cost, count/54000
    #return np.sum(ay.multiply(np.log(yh)))
  elif (ls_fun=='mse'):
    for i in range(len(ftrx[:54000])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost+=np.sum((ftry[i]- ypr)**2)
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return cost/54000, count/54000


def val_acc_loss(W,B,activation,ls_fun):
  cost=0
  count=0
  if (ls_fun=='ce'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost = np.sum(np.multiply(ftry[i],np.log(ypr)))
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return -cost, count/6000
    #return np.sum(ay.multiply(np.log(yh)))
  elif (ls_fun=='mse'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost+=np.sum((ftry[i]- ypr)**2)
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return cost/6000, count/6000


def test_acc(W,B,activation,ls_fun):
  count=0
  predictions=[]
  matrix=np.zeros((10,10))
  if (ls_fun=='ce'):
    for i in range(len(ftex)):
      ypr,_,_=Feed_Frwd_Nw1(ftex[i],W,B,activation,ls_fun)
      if (np.argmax(ypr) == ytest[i]):
        count+=1
      predictions.append(np.argmax(ypr))
      matrix[np.argmax(ypr)][ytest[i]]+=1
  lables={}
  for i in range(len(items)):
    lables[i]=items[i]
  wandb.log({"confusion matrix": wandb.plot.confusion_matrix(probs=None,
                                                              y_true=ytest,
                                                              preds=predictions,
                                                              class_names=items)})
  wandb.log({"confusion matrix v2.0": wandb.plots.HeatMap(items, items, matrix, show_text=True)})
  return count/len(ytest)

def Feed_Frwd_Nw1(xTr,Weighs,Bais,activation,loss):
  a_i=[]
  h_i=[]
  a_i.append(np.dot(Weighs[0],xTr)+Bais[0])       
  if (activation=='sig'):
    h_i.append(sigmoid(a_i[0]))
  elif (activation=='tanh'):
    h_i.append(Tanh(a_i[0]))
  elif (activation=='relu'):
    h_i.append(Relu(a_i[0]))
  for i in range(1,len(Weighs)-1):
    a_i.append((np.dot(Weighs[i],h_i[i-1])+Bais[i]))
    if (activation=='sig'):
      h_i.append(sigmoid(a_i[i]))
    elif (activation=='tanh'):
      h_i.append(Tanh(a_i[i]))
    elif (activation=='relu'):
      h_i.append(Relu(a_i[i]))
  a_i.append(np.dot(Weighs[-1],h_i[-1])+Bais[-1])
  if (loss=='ce'):
    h_i.append(softmax(a_i[-1]))
  elif (loss=='mse'):
    h_i.append(sigmoid(a_i[-1]))
  yp=h_i[-1]
  return yp,a_i,h_i


def Back_Prop(ip,ypr,ty,a_i,h_i,W,B,N,nrl,activation,Loss):
    k=len(ypr)
    dw,db,da,dh=initgrads(N,nrl)
    if (Loss=="ce"):
      dh[-1]=-(ty/ypr)
      da[-1]=ypr-ty
    elif (Loss=="mse"):
      dh[-1]=2*(ypr-ty)
      da[-1]=dh[-1]*grad_sigmoid(ypr)

    db[-1]=da[-1] 
    dw[-1]=np.dot((da[-1][:,np.newaxis]),(h_i[-2][:,np.newaxis]).T)
  
    for i in range(N-1,-1,-1):
      dh[i]=np.squeeze(np.dot(W[i+1].T,da[i+1]))

      if (activation=='sig'):
        da[i]=dh[i]*grad_sigmoid(a_i[i])
      elif (activation=='tanh'):
        da[i]=dh[i]*grad_Tanh(a_i[i])
      elif (activation=='relu'):
        da[i]=dh[i]*grad_Relu(a_i[i])
      db[i]=np.copy(da[i])
      if (i==0):
        dw[i]=np.dot(da[i][:,np.newaxis],ip[:,np.newaxis].T)
      else:
        dw[i]=np.dot(da[i][:,np.newaxis],h_i[i-1][:,np.newaxis].T)
    return dw,db

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
def gradDecent(ftrx,ftry,init,wd,eta,N,nrl,activation,Ls_fun,epochs,batchsize):
  W,B=InitParams(nrl,init)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          W[k]=np.subtract(W[k],eta*uw[k]/batchsize)
          B[k]=np.subtract(B[k],eta*ub[k]/batchsize)
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B

In [8]:
def rmsprop(ftrx,ftry,init,batchsize,eps,beta,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          vw[k]=np.add(beta*vw[k],(1-beta)*uw[k]**2)
          vb[k]=np.add(beta*vb[k],(1-beta)*ub[k]**2)
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*uw[k])
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*ub[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun)
    #wandb.log({"epoch":e,"Train_loss":LOSS/len(ftrx),"Train_acc":acc1/len(ftrx),"val_loss":testloss,"val_Accuracy":testacc})
    print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  #test_acc(W,B,activation,Ls_fun)
  return hi,W,B

In [9]:
nrl=[128] # number of neurons in each hidden layer
N=len(nrl)
hi,W,B=rmsprop(ftrx,ftry,'xaviour',32,1e-8,0.8,0.00005,0.25,N,nrl,'tanh','ce',epochs=3)

epochs : 1 train_loss : 0.008243925655433198   Train_acc: 0.8264814814814815 val_loss: 0.022521513184138815 val_Accuracy: 0.8301666666666667     time: 73
epochs : 2 train_loss : 0.009505330063839376   Train_acc: 0.844 val_loss: 0.05538573586772259 val_Accuracy: 0.8496666666666667     time: 72
epochs : 3 train_loss : 0.008216197819705874   Train_acc: 0.836 val_loss: 0.041778043652843634 val_Accuracy: 0.8365     time: 73
